## ML using XGBoost
Installations to run:

$ pip install --user xgboost

---------------------
CPU only

$ conda install -c conda-forge py-xgboost-cpu

--Or--

Use NVIDIA GPU

$ conda install -c conda-forge py-xgboost-gpu

In [1]:
# Dependencies
import seaborn as sns

import pandas as pd

import numpy as np

from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt

import warnings


warnings.filterwarnings("ignore")

In [2]:
#Load in dataset
df = pd.read_csv("data/test_1000.csv")

# Review the DataFrame
df.head()

,Unnamed: 0,Amount,Use Chip,Merchant State,Errors?,Is Fraud?,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,...,Has Chip,Cards Issued,International,Online,Age_at_transaction,income_to_debt,day_of_week,timestamp,time_of_day,distances
0,0,134.09,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,6,1.030861e+09,Morning,0.000000
1,1,38.48,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,6,1.030863e+09,Morning,33.540588
2,2,120.34,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,0,1.030948e+09,Morning,33.540588
3,3,128.95,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,0,1.030989e+09,Afternoon,33.540588
4,4,104.71,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,1,1.031034e+09,Morning,0.000000


In [3]:
df.shape

(11487390, 21)

In [4]:
df.describe()

,Unnamed: 0,Amount,Is Fraud?,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Has Chip,Cards Issued,International,Online,Age_at_transaction,income_to_debt,day_of_week,timestamp,distances
count,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.148739e+07,1.000386e+07
mean,6.066054e+06,5.270799e+01,1.214201e-03,2.454309e+04,4.766612e+04,5.982785e+04,7.092227e+02,3.673632e+00,8.960742e-01,1.522627e+00,6.491379e-03,1.226491e-01,4.589595e+01,2.162980e+06,3.003639e+00,1.340771e+09,1.657987e+02
std,3.508492e+06,7.597164e+01,3.482423e-02,1.251463e+04,2.501469e+04,5.263360e+04,6.805891e+01,1.569881e+00,3.051643e-01,5.179914e-01,8.030717e-02,3.280340e-01,1.562589e+01,1.033808e+07,1.999734e+00,1.594618e+08,6.050011e+02
min,0.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,4.880000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,2.008577e-01,0.000000e+00,6.628002e+08,0.000000e+00
25%,3.018895e+06,1.217000e+01,0.000000e+00,1.727100e+04,3.356600e+04,1.811600e+04,6.810000e+02,3.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.400000e+01,5.291839e-01,1.000000e+00,1.232008e+09,0.000000e+00
50%,6.067742e+06,3.414000e+01,0.000000e+00,2.163900e+04,4.146900e+04,5.241900e+04,7.140000e+02,4.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,4.400000e+01,7.622536e-01,3.000000e+00,1.357837e+09,0.000000e+00
75%,9.096439e+06,6.912000e+01,0.000000e+00,2.876000e+04,5.660500e+04,8.577100e+04,7.520000e+02,5.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.500000e+01,2.274873e+00,5.000000e+00,1.472105e+09,3.625127e+01
max,1.213659e+07,1.239050e+04,1.000000e+00,1.631450e+05,2.801990e+05,4.489290e+05,8.500000e+02,9.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,1.020000e+02,2.167400e+08,6.000000e+00,1.582934e+09,8.317257e+03


In [5]:
df.describe(exclude=np.number)

,Use Chip,Merchant State,Errors?,time_of_day
count,11487390,11487390,11487390,11487390
unique,3,191,23,4
top,Swipe Transaction,Online,No Error,Morning
freq,7159943,1408918,11302080,4612490


In [6]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = df.drop('Is Fraud?', axis=1), df[['Is Fraud?']]

In [7]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [8]:
X.dtypes

Unnamed: 0                        int64
Amount                          float64
Use Chip                       category
Merchant State                 category
Errors?                        category
Per Capita Income - Zipcode       int64
Yearly Income - Person            int64
Total Debt                        int64
FICO Score                        int64
Num Credit Cards                  int64
Has Chip                          int64
Cards Issued                      int64
International                     int64
Online                            int64
Age_at_transaction                int64
income_to_debt                  float64
day_of_week                       int64
timestamp                       float64
time_of_day                    category
distances                       float64
dtype: object

In [9]:
# Apply RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [10]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

In [11]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [12]:
# Define hyperparameters
params = {"objective": "binary:logistic", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [13]:
from sklearn.metrics import mean_squared_error

# Make predictions
train_preds = model.predict(dtrain_reg)
test_preds = model.predict(dtest_reg)

# Convert probabilities to binary predictions
train_preds_binary = (train_preds > 0.5).astype(int)
test_preds_binary = (test_preds > 0.5).astype(int)

In [14]:
rmse = mean_squared_error(y_test, test_preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")


RMSE of the base model: 0.180


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Evaluate with classification metrics
y_test_np = y_test.to_numpy()
print("Confusion Matrix (Test Set):")
print(confusion_matrix(y_test, test_preds_binary))
print("Classification Report (Test Set):")
print(classification_report(y_test, test_preds_binary))

Confusion Matrix (Test Set):
[[3342  197]
 [  99 3336]]
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.97      0.94      0.96      3539
           1       0.94      0.97      0.96      3435

    accuracy                           0.96      6974
   macro avg       0.96      0.96      0.96      6974
weighted avg       0.96      0.96      0.96      6974



In [16]:
print("Confusion Matrix (Training Set):")
print(confusion_matrix(y_train, train_preds_binary))
print("Classification Report (Training Set):")
print(classification_report(y_train, train_preds_binary))

Confusion Matrix (Training Set):
[[10223   186]
 [   18 10495]]
Classification Report (Training Set):
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     10409
           1       0.98      1.00      0.99     10513

    accuracy                           0.99     20922
   macro avg       0.99      0.99      0.99     20922
weighted avg       0.99      0.99      0.99     20922

